In [1]:
import numpy as np
import math

In [9]:
# arrival = 5
# P0 = np.array([0.5, 0.5, 0])
# P = np.array([[0, 0.4, 0.6], [0, 0, 0], [0, 0, 0]])
# muy = np.array([15, 12, 10])



arrival = 100000
P0 = np.array([0.4, 0.3, 0.3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

P = np.zeros((13, 13))
P[0, 3] = 0.9
P[0, 4] = 0.1
P[1, 3] = 0.9
P[1, 4] = 0.1
P[2, 3] = 0.9
P[2, 4] = 0.1
P[3, 0] = 0.01
P[3, 1] = 0.01
P[3, 2] = 0.01
P[3, 5] = 0.155
P[3, 6] = 0.155
P[3, 7] = 0.155
P[3, 8] = 0.155
P[3, 9] = 0.155
P[3, 10] = 0.155
P[4, 0] = 0.01
P[4, 1] = 0.01
P[4, 2] = 0.01
P[4, 11] = 0.9
P[5, 12] = 0.02
P[6, 12] = 0.02
P[7, 12] = 0.02
P[8, 12] = 0.02
P[9, 12] = 0.02
P[10, 12] = 0.02
P[11, 12] = 0.02


muy = np.array([50000, 50000, 50000, 100000, 30000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 5000])
a = arrival * P0
num_server_mmcqueue = 3


#For 4 server check info and 7 server check sercurity
# num_server_mmcqueue = 4
# P = np.zeros((14, 14))
# P[3, 5] = 0.132
# P[3, 6] = 0.132
# P[3, 7] = 0.132
# P[3, 8] = 0.132
# P[3, 9] = 0.132
# P[3, 10] = 0.132
# P[3, 13] = 0.138
# P[13, 12] = 0.02

In [3]:
class MMCQueue:
    def __init__(self, lamda, muy, number_servers):
        self.lamda = lamda
        self.muy = muy
        self.number_servers = number_servers
        self.utilization = lamda / (number_servers * muy)
        self.probability_arrival = caculate_probability_MMCqueue(self.number_servers, self.utilization)
        self.mean_number_of_jobs = (self.utilization / (1 - self.utilization))*self.probability_arrival + (number_servers*self.utilization)
        self.response_time = (self.probability_arrival/((self.number_servers*self.muy)-self.lamda))+(1/self.muy)
        self.waiting_time = self.response_time - (1/self.muy)

In [4]:
class MM1Queue:
    def __init__(self, lamda, muy):
        self.lamda = lamda
        self.muy = muy
        self.utilization = lamda / muy
        self.mean_number_of_jobs = self.utilization / (1 - self.utilization)
        self.variance_number_of_jobs = self.utilization / pow((1 - self.utilization), 2)
        self.mean_system_time = self.mean_number_of_jobs / lamda
        self.mean_waiting_time = self.mean_system_time - 1/muy

In [5]:
def caculate_probability_MMCqueue(number_servers, utilization):
    sum = 0
    for i in range(number_servers):
      sum = sum + (pow(number_servers * utilization, i) / math.factorial(i))
    return 1 / (1 + (1 - utilization) * (math.factorial(number_servers) / pow(number_servers * utilization, number_servers)) * sum)

In [6]:
def round_matrix(matrix, decimal_places):
    rounded_matrix = []
    for row in matrix:
        rounded_row = [round(element, decimal_places) for element in row]
        rounded_matrix.append(rounded_row)
    return rounded_matrix

In [7]:
def inv_ip(P_func, mi_func):
  n = P_func.shape[0]
  I = np.identity(n)
  lamda = np.dot(np.linalg.inv(I - P_func.T), a)
  return lamda


In [10]:
def main():
  #Test MMC
  # temp = MMCQueue(500, 300, 3)
  # print("--------------------")
  # print("THIS IS M/M/C QUEUE, STT: {index + 1}")
  # print("lamda:", temp.lamda)
  # print("muy:", temp.muy)
  # print("number_servers:",temp.number_servers)
  # print("utilization:", temp.utilization)
  # print("mean_number_of_jobs:", temp.mean_number_of_jobs)
  # print("probability_arrival:", round(temp.probability_arrival,5))
  # print("mean_system_time:", temp.response_time)
  # print("mean_waiting_time:", temp.waiting_time)



  lamda = inv_ip(P, muy)

  print("Arrival rate λ:")
  print(lamda)

  print("Service rate µ:")
  print(muy)

  utilization = np.divide(lamda, muy)
  print("Utilization:")
  print(utilization)

  mean_num_jobs_system = 0
  mean_system_time = 0
  mean_waiting_time = 0

  queues = []
  for i in range(13):
    if (i==3):
      queue = MMCQueue(lamda[i],muy[i],3)
      queues.append(queue)
      mean_num_jobs_system = mean_num_jobs_system + queue.mean_number_of_jobs
      mean_system_time = mean_system_time + queue.response_time
      mean_waiting_time = mean_waiting_time + queue.waiting_time
    else:
      queue = MM1Queue(lamda[i], muy[i])
      queues.append(queue)
      mean_num_jobs_system = mean_num_jobs_system + queue.mean_number_of_jobs
      mean_system_time = mean_system_time + queue.mean_system_time
      mean_waiting_time = mean_waiting_time + queue.mean_waiting_time

  print("Mean number of jobs in system: ")
  print(round(mean_num_jobs_system, 2))

  print("Mean system time in network queue: ")
  print(round(mean_num_jobs_system / arrival,5))


  #print("Mean waiting time in network queue: ")
  #print(round(mean_waiting_time, 5))


  # for index, queue in enumerate(queues):
  #    print("--------------------")
  #    print(f'THIS IS M/M/1 QUEUE, STT: {index + 1}')
  #    print("lamda:", queue.lamda)
  #    print("muy:", queue.muy)
  #    print("utilization:", round(queue.utilization,2))
  #    print("mean_number_of_jobs:", round(queue.mean_number_of_jobs, 2))
  #    print("variance_number_of_jobs:", round(queue.variance_number_of_jobs,2))
  #    print("mean_system_time:", round(queue.mean_system_time,5))
  #    print("mean_waiting_time:", round(queue.mean_waiting_time,5))

  for index, queue in enumerate(queues):
    if (queue.muy == muy[3]):
      print("--------------------")
      print(f'THIS IS M/M/C QUEUE, STT: {index + 1}')
      print("lamda:", queue.lamda)
      print("muy:", queue.muy)
      print("number_servers:",queue.number_servers)
      print("utilization:", round(queue.utilization,2))
      print("mean_number_of_jobs:", round(queue.mean_number_of_jobs))
      print("probability_arrival:", round(queue.probability_arrival,5))
      print("response_time:", round(queue.response_time,5))
      print("mean_waiting_time:", round(queue.waiting_time,6))
    else:
      print("--------------------")
      print(f'THIS IS M/M/1 QUEUE, STT: {index + 1}')
      print("lamda:", queue.lamda)
      print("muy:", queue.muy)
      print("utilization:", round(queue.utilization,2))
      print("mean_number_of_jobs:", round(queue.mean_number_of_jobs, 2))
      print("variance_number_of_jobs:", round(queue.variance_number_of_jobs,2))
      print("mean_system_time:", round(queue.mean_system_time,5))
      print("mean_waiting_time:", round(queue.mean_waiting_time,5))

if __name__ == "__main__":
  main()

Arrival rate λ:
[ 39062.5   39062.5   39062.5   39062.5  140625.    15625.    18675.
  18675.    18675.    18675.    18675.    18675.    18675.    14062.5
   2895.75]
Service rate µ:
[ 50000  50000  50000  50000 150000  30000  25000  25000  25000  25000
  25000  25000  25000  25000   5000]
Utilization:
[0.78125    0.78125    0.78125    0.78125    0.9375     0.52083333
 0.747      0.747      0.747      0.747      0.747      0.747
 0.747      0.5625     0.57915   ]
Mean number of jobs in system: 
39.65
Mean system time in network queue: 
0.00026
--------------------
THIS IS M/M/1 QUEUE, STT: 1
lamda: 39062.5
muy: 50000
utilization: 0.78
mean_number_of_jobs: 3.57
variance_number_of_jobs: 16.33
mean_system_time: 9e-05
mean_waiting_time: 7e-05
--------------------
THIS IS M/M/1 QUEUE, STT: 2
lamda: 39062.5
muy: 50000
utilization: 0.78
mean_number_of_jobs: 3.57
variance_number_of_jobs: 16.33
mean_system_time: 9e-05
mean_waiting_time: 7e-05
--------------------
THIS IS M/M/1 QUEUE, STT: 3
lam